In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Internships/Textile/'

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import shutil
import glob
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [3]:
baseModel = VGG16(weights="imagenet", include_top = False,input_tensor=Input(shape=(224,224,3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
rescale = 1./255
target_size = (224, 224)
batch_size = 32
class_mode = "categorical"


train_datagen = ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory('gdrive/My Drive/Internships/Textile/Textile_Defect_train/',
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)

validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory('gdrive/My Drive/Internships/Textile/Textile_Defect_test/',
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle = False)

Found 256 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


In [12]:
steps_per_epoch=len(train_generator)
validation_steps=len(validation_generator)
loss='categorical_crossentropy'
metrics=['accuracy']
epochs = 10

opt = Adam(lr=1e-3, decay=1e-3 / 10)
model.compile(optimizer=opt, loss=loss, metrics=metrics)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=epochs,
    verbose=1,
  #  callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=validation_steps 
 #   class_weight=class_weight
    )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
8/8 [==============================] - 146s 19s/step - loss: 0.7252 - accuracy: 0.5195 - val_loss: 0.6193 - val_accuracy: 0.7344
Epoch 2/10
8/8 [==============================] - 142s 18s/step - loss: 0.6541 - accuracy: 0.6172 - val_loss: 0.5966 - val_accuracy: 0.7656
Epoch 3/10
8/8 [==============================] - 142s 18s/step - loss: 0.6082 - accuracy: 0.6758 - val_loss: 0.5831 - val_accuracy: 0.7656
Epoch 4/10
8/8 [==============================] - 143s 18s/step - loss: 0.5763 - accuracy: 0.6953 - val_loss: 0.5730 - val_accuracy: 0.7656
Epoch 5/10
8/8 [==============================] - 142s 18s/step - loss: 0.5623 - accuracy: 0.7031 - val_loss: 0.5622 - val_accuracy: 0.7656
Epoch 6/10
8/8 [==============================] - 143s 18s/step - loss: 0.5626 - accuracy: 0.7500 - val_loss: 0.5555 - val_accuracy: 0.7656
Epoch 7/10
8/8 [==============================] - 142s 18s/step - loss: 0.5512 - accuracy: 0.7344 - val_loss: 0.5480 - val_accuracy: 0.7969
Epoch 8/10
8/8 [====

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

print("results for training")
result  = model.evaluate_generator(train_generator, verbose=1)
print("%s%.2f  "% ("Loss     : ", result[0]))
print("%s%.2f%s"% ("Accuracy : ", result[1]*100, "%"))
classes = os.listdir('gdrive/My Drive/Internships/Textile/Textile_Defect_train/')


print("results for test")
result  = model.evaluate_generator(validation_generator, steps=len(validation_generator), verbose=1)
print("%s%.2f  "% ("Loss     : ", result[0]))
print("%s%.2f%s"% ("Accuracy : ", result[1]*100, "%"))
print("results for test prediction")

results for training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  after removing the cwd from sys.path.


8/8 [==============================] - 114s 14s/step - loss: 0.5100 - accuracy: 0.7773
Loss     : 0.51  
Accuracy : 77.73%
results for test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


2/2 [==============================] - 28s 14s/step - loss: 0.5238 - accuracy: 0.7656
Loss     : 0.52  
Accuracy : 76.56%
results for test prediction


In [14]:
y_pred = model.predict_generator(validation_generator, steps=len(validation_generator), verbose=1)  
y_pred = y_pred.argmax(axis=-1)
y_true=validation_generator.classes
precision = precision_score(y_true, y_pred) 
recall = recall_score(y_true, y_pred) 
f1 = f1_score(y_true, y_pred) 
print("-"*90)
print("Derived Report")
print("-"*90)
print("%s%.2f%s"% ("Precision     : ", precision*100, "%"))
print("%s%.2f%s"% ("Recall        : ", recall*100,    "%"))
print("%s%.2f%s"% ("F1-Score      : ", f1*100,        "%"))
print("-"*90)
print("\n\n")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


2/2 [==============================] - 28s 14s/step
------------------------------------------------------------------------------------------
Derived Report
------------------------------------------------------------------------------------------
Precision     : 87.50%
Recall        : 51.85%
F1-Score      : 65.12%
------------------------------------------------------------------------------------------



